In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import csv
import re
import os
from cobra.flux_analysis import pfba

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [17]:
model = cobra.io.read_sbml_model('/home/dengxiao/mqc/mqc/local_test_data/other/iZM516_20230711_bigg.sbml')
general_library = cobra.io.read_sbml_model('/home/dengxiao/mqc/mqc/local_test_data/other/model_general_library.xml')

In [3]:
general_library.metabolites.get_by_id('13ppd_c0')

Metabolite identifier,13ppd_c0
Name,"1,3-Propanediol"
Memory address,0x7f70cdabf1c0
Formula,C3H8O2
Compartment,c
In 2 reaction(s),"rxn26410, rxn02235"


In [4]:
def gapfilling(model, demand_id, general_library):
    """"""
    with model:
        for rxns in model.reactions:  # 给系数扩大1000倍,添加的demand反应不改
            temp_dict = {}
            for met in rxns.metabolites:
                temp_dict[met] = rxns.get_coefficient(met) * (1000)
            rxns.add_metabolites(temp_dict) # 相同时会覆盖掉原代谢物。乘以1000是为了pfba计算
        for rxns in general_library.reactions:
            if rxns.id not in model.reactions and rxns.id != 'GLCS1' and 'BIOMASS' not in rxns.id and 'bio1' not in rxns.id and len(rxns.products) != 0:
                model.add_reaction(rxns)  
        #cobra.io.write_sbml_model(model,'model_general_library.xml')
        reaction = Reaction(demand_id) 
        model.add_reactions([reaction])
        reaction.build_reaction_from_string(demand_id + ' --> ')
        model.reactions.get_by_id(demand_id).bounds = (0,1)  #dna （0，1） 
        model.objective = demand_id
        model.reactions.get_by_id('EX_glc_D_e').bounds = (-1000,0)  #dna （0，1）
        pfba_solution = pfba(model)  # 将模型和总库的反应融合到一起后，再次计算当前目标下的通量结果文件；随后把生成目标的途径添加到模型中
        need_fluxes = pfba_solution.fluxes[abs(pfba_solution.fluxes)>1e-6]
        new_way_rxn = list(need_fluxes.keys())
    return new_way_rxn

In [18]:
demand_id = '13ppd_c0'
need_add_rea=gapfilling(model, demand_id, general_library)

将需要gap的反应添加到原模型，然后再次以biomass为目标求解途径

In [20]:
model_rea,need_add_extra_rxn=[],[]

for rxnId in need_add_rea:
    if rxnId not in model.reactions and rxnId != demand_id:
        need_add_extra_rxn.append(rxnId)
        print(rxnId)

with model:
    for rxnId in need_add_extra_rxn:
        rxns = general_library.reactions.get_by_id(rxnId)
        model.add_reaction(rxns)
    pfba_solution = pfba(model)  # 将模型和总库的反应融合到一起后，再次计算当前目标下的通量结果文件；随后把生成目标的途径添加到模型中
    need_fluxes = pfba_solution.fluxes[abs(pfba_solution.fluxes)>1e-6]
    flux_table = f'{demand_id}.txt'
    with open(flux_table, 'w') as flux:  
        for r,v in need_fluxes.items():
            rxns = model.reactions.get_by_id(r) 
            try:
                check = rxns.check_mass_balance()  
                flux.write(f"{r}\t{round(v,5)}\t{rxns.reaction}\t{rxns.build_reaction_string(use_metabolite_names=True)}\t{rxns.bounds}\t{check}\n") 
            except:
                flux.write(f"{r}\t{round(v,5)}\t{rxns.reaction}\t{rxns.build_reaction_string(use_metabolite_names=True)}\t{rxns.bounds}\tno_check_mass_balance\n") 

rxn00768
rxn26410
rxn00617


In [27]:
for i in model.reactions:
    a=model.reactions.get_by_id(i.id).lower_bound
    if not a==0 and not a==-1000:
        print(i.id, i.reaction,i.bounds)


ATPM atp_c0 + h2o_c0 --> adp_c0 + h_c0 + pi_c0 (3.71, 3.71)
EX_glc_D_e glc__D_e0 <=>  (-40.0, 1000.0)


In [4]:
def add_gapfilling_rxn(self, model, model_info, demand_id, general_library):
        new_way_rxn = self.gapfilling(model, demand_id, general_library)
        num, need_add_rxn = 0, []
        if not new_way_rxn:
            gap_info = ["Not able to gapfill the response!!!"]
        else:
            gap_info = ["gapfilling added reaction:"]
            for rxnId in new_way_rxn:
                if rxnId not in model.reactions and rxnId in general_library.reactions:
                    num += 1
                    rxns = general_library.reactions.get_by_id(rxnId)
                    model.add_reaction(rxns)
                    need_add_rxn.append(rxnId)
                    rxn_exp = [f"{rxns}"]
                    if model_info['model_identifier'] == 'modelseed' or model_info['model_identifier'] == 'metaNetx':
                        rxn_exp = [f"{rxns.id}: {rxns.build_reaction_string(use_metabolite_names=True)}"]
                    gap_info.extend(rxn_exp)
            if num == 0:
                gap_info.extend(["no gap response,since it already exists!"])
        return need_add_rxn, gap_info

In [5]:
def add_gap_rxn(self, model, need_add_rxn, general_library):
        """"""
        for rxnId in need_add_rxn:
            rxns = general_library.reactions.get_by_id(rxnId)
            model.add_reaction(rxns)

In [6]:
def gap_initial_zero_bio(self, model_info, model, model_control_info, initial_rxn_id, general_library):
        """"""
        no_synthesized_mets, num = [], 0
        temp = [f"Biomass initial rate is 0, check precursor components"]
        for met in model.reactions.get_by_id(initial_rxn_id).reactants:
            num += 1
            with model:
                objectiveId = add_demand(model_info, model, str(met.id))
                model.objective = objectiveId
                ini_flux = model.slim_optimize()
                if pd.isna(ini_flux) or ini_flux <= 1e-6:
                    temp.extend([f"The {str(met.id)} rate of the biomass component material is 0, and try to gap:"])
                    no_synthesized_mets.append(str(met.id))
                    need_add_rxn, gap_info = self.add_gapfilling_rxn(big_model, model_info, objectiveId, general_library, 'pre')
                    temp.extend(gap_info)    
                    self.add_gap_rxn(model, need_add_rxn, general_library)
        write_biomass(model_control_info, temp, "预处理")
        return need_add_rxn, no_synthesized_mets	

In [8]:
import cobra 
# general_library = cobra.io.read_sbml_model("mqc/summary/seed_1.xml")
general_library = cobra.io.read_sbml_model('D:/User/takura/Desktop/mqc.xml')
general_library.reactions.get_by_id('Add_ATP_maintain')

KeyError: 'Add_ATP_maintain'

In [9]:
general_library.reactions.remove('Add_ATP_maintain')
general_library.reactions.get_by_id('Add_ATP_maintain')

ValueError: Add_ATP_maintain not found